Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
            [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [17]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.376161
Minibatch accuracy: 6.2%
Validation accuracy: 9.0%
Minibatch loss at step 50: 2.034088
Minibatch accuracy: 31.2%
Validation accuracy: 43.1%
Minibatch loss at step 100: 0.985778
Minibatch accuracy: 68.8%
Validation accuracy: 63.2%
Minibatch loss at step 150: 1.225320
Minibatch accuracy: 75.0%
Validation accuracy: 73.9%
Minibatch loss at step 200: 0.856515
Minibatch accuracy: 68.8%
Validation accuracy: 77.4%
Minibatch loss at step 250: 0.846469
Minibatch accuracy: 62.5%
Validation accuracy: 76.9%
Minibatch loss at step 300: 0.535842
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step 350: 0.911263
Minibatch accuracy: 68.8%
Validation accuracy: 79.0%
Minibatch loss at step 400: 0.967623
Minibatch accuracy: 75.0%
Validation accuracy: 78.8%
Minibatch loss at step 450: 0.491029
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Minibatch loss at step 500: 0.627737
Minibatch accuracy: 81.2%
Validation accuracy: 80.4%
Min

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [18]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
            [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        hidden = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                                padding='SAME')
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        hidden = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                                padding='SAME')
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [19]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.962266
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.225925
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 100: 1.833439
Minibatch accuracy: 25.0%
Validation accuracy: 42.0%
Minibatch loss at step 150: 1.628520
Minibatch accuracy: 56.2%
Validation accuracy: 60.2%
Minibatch loss at step 200: 0.789304
Minibatch accuracy: 81.2%
Validation accuracy: 75.0%
Minibatch loss at step 250: 0.832132
Minibatch accuracy: 81.2%
Validation accuracy: 77.2%
Minibatch loss at step 300: 0.466263
Minibatch accuracy: 87.5%
Validation accuracy: 79.5%
Minibatch loss at step 350: 0.850434
Minibatch accuracy: 75.0%
Validation accuracy: 78.8%
Minibatch loss at step 400: 1.233523
Minibatch accuracy: 62.5%
Validation accuracy: 78.4%
Minibatch loss at step 450: 0.517089
Minibatch accuracy: 81.2%
Validation accuracy: 82.4%
Minibatch loss at step 500: 0.601813
Minibatch accuracy: 87.5%
Validation accuracy: 82.5%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

Solution
----
Note: The codes are refectored to make them more manageable similar to the one given [here](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/convolutional_network.py)

#### Build a multilayer convolutional neural networks

* Input and Network Parameters

In [6]:
#Parameters
image_size=28
n_channels = 1
batch_size = 128
patch_size = 5
learning_rate = 1e-4
num_steps = 5000
display_step = 100

#Network Parameters
n_inputs = image_size**2 # 784
n_depth1 = 32
n_depth2 = 64
#n_depth3 = 128
n_depth3 = 1024
n_classes = 10 # Number of output classes (A-J)

dropout = 0.75  # Dropout for regularization

* Placeholders

In [7]:
x = tf.placeholder(tf.float32, shape=[None, n_inputs])
y = tf.placeholder(tf.float32, shape=[None, n_classes])
keep_prob = tf.placeholder(tf.float32)

* Simple Wrappers for constructing the network

In [8]:
# 1. Weight and Bias Initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 2. Convolution and Pooling
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

# 3. Model Construction function (2 Conv. layers and 1 fully connected layer)
def model(x, weights, biases, dropout):
    # Reshape the input data
    x = tf.reshape(x, [-1,28,28,1])

    # 1st convolutional layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # 2nd Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wf1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wf1']), biases['bf1'])
    fc1 = tf.nn.relu(fc1)

    # Apply Dropout (To reduce overfitting)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Readout Layer (for class prediction)
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out 

* Initialize weights and bias for the tensor graph

In [9]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': weight_variable([patch_size, patch_size, n_channels, n_depth1]),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': weight_variable([patch_size, patch_size, n_depth1, n_depth2]),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wf1': weight_variable([(image_size//4)*(image_size//4)*n_depth2, n_depth3]),
    # 1024 inputs, 10 outputs (class prediction)
    'out': weight_variable([n_depth3, n_classes])
}

biases = {
    'bc1': bias_variable([n_depth1]),
    'bc2': bias_variable([n_depth2]),
    'bf1': bias_variable([n_depth3]),
    'out': bias_variable([n_classes])
}

* Start TensorFlow InteractiveSession

In [10]:
sess = tf.InteractiveSession()

* Reshape the data to be compatible with the model input format

In [11]:
train_dataset = train_dataset.reshape((-1, n_inputs))
valid_dataset = valid_dataset.reshape((-1, n_inputs))
test_dataset = test_dataset.reshape((-1, n_inputs))

* Train and Evaluate the Model

In [12]:
y_ = model(x, weights, biases, keep_prob)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_, y))

# Optimizer: set up a variable that's incremented once per batch and
# controls the learning rate decay.
#global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
#learning_rate = tf.train.exponential_decay(0.5, global_step, 10000, 0.95, staircase=True)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, 
#             global_step=global_step)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

* Inititialize the session variables

In [13]:
sess.run(tf.initialize_all_variables())

* Run the session, optimize and evaluate the model

In [14]:
print('Initialized')
for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_x = train_dataset[offset:(offset + batch_size), :]
    batch_y = train_labels[offset:(offset + batch_size), :]
    
    sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

    if step%display_step == 0:
        l, acc = sess.run([loss, accuracy], feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        print("Step: %d, Loss: %.4f, Training accuracy: %.2f"%(step, l, acc))
        
        acc = accuracy.eval(feed_dict={x: valid_dataset, y: valid_labels, keep_prob: 1.0})
        print("Validation accuracy: %.2f"%(acc))

print("Test accuracy %g"%accuracy.eval(feed_dict={
            x: test_dataset, y: test_labels, keep_prob: 1.0}))

Initialized
Step: 0, Loss: 5.8070, Training accuracy: 0.16
Validation accuracy: 0.14
Step: 100, Loss: 0.9946, Training accuracy: 0.77
Validation accuracy: 0.78
Step: 200, Loss: 0.6617, Training accuracy: 0.85
Validation accuracy: 0.81
Step: 300, Loss: 0.4517, Training accuracy: 0.85
Validation accuracy: 0.83
Step: 400, Loss: 0.6966, Training accuracy: 0.82
Validation accuracy: 0.84
Step: 500, Loss: 0.5394, Training accuracy: 0.85
Validation accuracy: 0.85
Step: 600, Loss: 0.5640, Training accuracy: 0.85
Validation accuracy: 0.85
Step: 700, Loss: 0.5817, Training accuracy: 0.83
Validation accuracy: 0.86
Step: 800, Loss: 0.3780, Training accuracy: 0.88
Validation accuracy: 0.86
Step: 900, Loss: 0.3990, Training accuracy: 0.87
Validation accuracy: 0.87
Step: 1000, Loss: 0.6494, Training accuracy: 0.78
Validation accuracy: 0.87
Step: 1100, Loss: 0.4974, Training accuracy: 0.84
Validation accuracy: 0.87
Step: 1200, Loss: 0.4779, Training accuracy: 0.85
Validation accuracy: 0.87
Step: 1300, 